# Метод внешних штрафов

In [32]:
import numpy as np
import numpy.linalg as lg

## Функция и ее градиент

In [33]:
def fx(x):
    return x[0] ** 2 + x[1] ** 2 - 10 * x[0] - 8 * x[1]

def gradfx(x):
    arr = np.array([2 * x[0] - 10, 
                    2 * x[1] - 8, 
                    0 
                   ])
    return arr;

## Условие и его градиент

In [34]:
def Hx(x):
    return (max(0, -x[0])) ** 2 + (max(0, -x[1])) ** 2 + (max(0, -x[2])) ** 2 + (2 * x[0] + 3 * x[1] + x[2] - 6) ** 2

def gradHx(x):
    arr = np.array([- 2 * max(0, -x[0]) + 4 * (2 * x[0] + 3 * x[1] + x[2] - 6),
                    - 2 * max(0, -x[1]) + 6 * (2 * x[0] + 3 * x[1] + x[2] - 6),
                    - 2 * max(0, -x[2]) + 2 * (2 * x[0] + 3 * x[1] + x[2] - 6)
                    ])
    return arr;

## вспомогательные функции, которые не стоит использовать

In [35]:
def mygeom(x0, e, grad):
    ##print('    start mygeom')
    a = 0
    scalar = abs(sum(grad(x0)*grad(x0 - a * grad(x0))))
    while scalar > e: 
        a+=e
        scalar1 = scalar
        scalar = abs(sum(grad(x0)*grad(x0 - a * grad(x0))))
        #print('    a = ', a)#print('    scalar = ', scalar)
        if scalar1 < scalar:
            #print('    break', scalar)#print('        break', scalar) #print('        break', scalar1)
            break
    ##print('    a = ', a)
    ##print('    scalar = ', scalar)
    ##print('    end mygeom')
    return a
#------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------------------#
def func(f, x0, gradx0):
    def fun(a):
        A = x0 -  gradx0 * a
        return f(A)
    return fun

# первая производная
#------------------------------------------------------------------------------------------------------------#
def der1_e(f_, x0, a, gradx0, e):
    e = e / 2
    f_(x0)
    f = func(f_, x0, gradx0)
    def der(a):
        return (f(a + e) - f(a - e)) / (e * 2)
    return der
#------------------------------------------------------------------------------------------------------------#
# вторая производная
#------------------------------------------------------------------------------------------------------------#
def der2_e(f_, x0, a, gradx0, e):
    e = e / 2
    df = der1_e(f_, x0, a, gradx0, e)
    def der2(a):
        return (df(a + e) - df(a - e)) / ( e * 2 )
    return der2
#------------------------------------------------------------------------------------------------------------#
# ищем а при одномерной минимизации по направлению
#------------------------------------------------------------------------------------------------------------#
def grad_descent_a1(f, x0, x1, e):
    a = 0
    df = der1_e(f, x0, a, x1-x0, e / 5)
    while e < abs(df(a)):
        d2f = der2_e(f, x0, a, x1-x0, e / 5)
        a = a - df(a)/d2f(a)
        df = der1_e(f, x0, a, x1-x0, e / 5)
    return a
#------------------------------------------------------------------------------------------------------------#
# тоже самое, только на вход подается начало вектора и сам вектор, а не его начало и конец
#------------------------------------------------------------------------------------------------------------#
def grad_descent_a2(f, x0, vec, e):
    a = 0
    df = der1_e(f, x0, a, vec, e / 5)
    while e < abs(df(a)):
        d2f = der2_e(f, x0, a, vec, e / 5)
        a = a - df(a)/d2f(a)
        df = der1_e(f, x0, a, vec, e / 5)
    return a

## получение одномерной функции и золотое сечение

In [36]:
def func0(f, x0, gradx0):
    def fun(a):
        A = x0 -  gradx0 * a
        return f(A)
    return fun
#------------------------------------------------------------------------------------------------------------#
#золотое сечение
#------------------------------------------------------------------------------------------------------------#
def golden_section(f, a_, b_, e):
    n = 0
    a = a_
    b = b_
    x = a
    c = (3 - 5 ** 0.5 ) / 2 * (b - a) + a
    d = (5 ** 0.5 - 1) / 2 * (b - a) + a
    while (b - a) >= 2 * e:
        n+=1
        yc = f(c)
        yd = f(d)
        if(yc < yd):
            b = d
            d = c
            c = (3 - 5 ** 0.5 ) / 2 * (b - a) + a
        else:
            a = c
            c = d
            d = (5 ** 0.5 - 1) / 2 * (b - a) + a
    
    '''
    if (n%10 == 0) or (20 > n > 4) or ( n % 10 > 4):
        print('сделано ', n, 'шагов')
    elif(5 > n % 10 > 1):
        print('сделано ', n, 'шага')
    elif (n % 10 == 1):
        print('сделан ', n, 'шаг')
    '''
    return a

In [37]:
#------------------------------------------------------------------------------------------------------------#
# golden_section --- 
#------------------------------------------------------------------------------------------------------------#
def grad_descent_a01(f_, x0, x1, e):
    f = func0(f_, x0, x1-x0)
    a = golden_section(f, -5., 5., e/10)
    return a
#------------------------------------------------------------------------------------------------------------#
# golden_section --- начало вектора и сам вектор
#------------------------------------------------------------------------------------------------------------#
def grad_descent_a02(f_, x0, vec, e):
    f = func0(f_, x0, vec)
    a = golden_section(f, -5., 5., e/10)
    return a

## МНГС

In [38]:
def min_gradient(f, grad, x0, e):
    n = 0
    while abs(lg.norm(grad(x0))) > e:
        n+=1
        print('')
        print('------[----', n, '----]------')
        #a = mygeom(x0, e/1000, grad)
        a = grad_descent_a02(f, x0, grad(x0), e/10)
        print('')
        print('a = ', a)
        print('  x  = ', x0)
        print('f(x) = ', f(x0))
        print(' |grad(x)| = ', lg.norm(grad(x0)))
        x0 = x0 - a * grad(x0)
    print('сделано ', n, ' шагов')
    return x0

### Ускоренный градиентный метод p-того порядка

In [39]:
#------------------------------------------------------------------------------------------------------------#
def p_gradient(f, grad, x0, e):
    n  = 0
    n1 = 0;
    while abs(lg.norm(grad(x0))) > e:
        x1 = x0
        
        for k in range(len(x0) * 4):
            if(abs(lg.norm(grad(x1))) <= e):
                x0 = x1
                break
            n+=1
            print('')
            print('------[----', n, '----]------')
            #a = mygeom(x1, e/1000, grad)
            a = grad_descent_a02(f, x1, grad(x1), e / 10)
            x1 = x1 - grad(x1) * a

            print('')
            print('a = ', a)
            print('  x  = ', x1)
            print('f(x) = ', f(x1))
            if f(x1) >= f(x0):
                break
        if(abs(lg.norm(grad(x0))) <= e):
            break
        if f(x1) >= f(x0):
            break
        print('-------------------------------------------------------------------')
        print('----------------------------скачок[----', n1, '----]скачок')
        a = grad_descent_a1(f, x0, x1, e / 100)
        x0 = x0 -  (x1 - x0) * a
        print('----------------------------  a1  = ', a)
        print('----------------------------  x1  = ', x0)
        print('----------------------------f(x1) = ', f(x0))
        n1+=1
        
    print('сделано ', n, ' шагов')
    print('сделано ', n1, ' скачков')
    return x0
#------------------------------------------------------------------------------------------------------------#

#------------------------------------------------------------------------------------------------------------#
def ravine_gradient(f, grad, x0, e):
    x1 = x0 - e*10
    n  = 0
    n1 = 0
    while abs(lg.norm(grad(x0))) > e:
        for k in range(len(x0)):
            if(abs(lg.norm(grad(x0))) < e):
                break
            n+=1
            print('')
            print('------[----', n, '----]------')
            
            #a = mygeom(x0, e/100, grad)
            a = grad_descent_a02(f, x1, grad(x1), e / 10)
            x0 = x0 - grad(x0) * a
            
            #a = mygeom(x1, e/100, grad)
            a1 = grad_descent_a02(f, x1, grad(x1), e / 10)
            x1 = x1 - grad(x1) * a1
            
            print('  x0    = ', x0 ,   ' x1    = ', x1 )
            print('  f(x0) = ', f(x0), ' f(x1) = ', f(x1) )
            
        if(abs(lg.norm(grad(x0))) < e):
                break
                
        print('')
        print('скачок[----', n1, '----]скачок')
        a = grad_descent_a1(f, x0, x1, e / 100)
        x0 = x0 - a * (x1-x0)
        x1 = x0 - e*10
        print('a1 = ', a)
        print('  x1  = ', x0)
        print('f(x1) = ', f(x0))
        n1+=1
        
    print('сделано ', n, ' шагов')
    print('сделано ', n1, ' скачков')
    return x0
#------------------------------------------------------------------------------------------------------------#

## Реализаия метода

### Суммарная функция *f(x)+rH(x)*

In [40]:
def sumf(f, r , H):
    def result(x):
        res = f(x) + r * H(x)
        return res
    return result

In [44]:
def r_0(n):
    return 10 ** n
def r_1(n):
    return np.sqrt(n)
def r_2(n):
    return ((n * 2 + 10) ** (1/2) + 10)
def r_3(n):
    return ((n * 2 + 5 ) ** (1/2) + 5 )
def r_4(n):
    return ((n * 2 + 6 ) ** (1/2) + 6 )
def r_active(n):
    return ((n * 3 + 10) ** (1/2) + 10)

def sanction( f_, gradf_, H, gradH, x, e):
    n = 1
    r = r_active(n)
    f = sumf(f_, r, H)
    grad = sumf(gradf_, r, gradH)
    
    while( H(x) > e) or (lg.norm(grad(x)) > e):
        print('------------------------------------------------------------------------[----', n, '----]------')
        #x = min_gradient(f, grad, x, e)#лучше разделить на 10
        #x = p_gradient(f, grad, x, e)
        x = ravine_gradient(f, grad, x, e)
        n += 1
        r = r_active(n)
        f = sumf(f_, r, H)
        grad = sumf(gradf_, r, gradH)
        
        print(' -------------------------------------------------------------- f(x) + (( ', r ,' )) * H(x)    =    ', f(x))
        print(' --------------------------------------------------------------   x       = ', x)
        print(' -------------------------------------------------------------- H(x)      = ', H(x))
        print(' -------------------------------------------------------------- |grad(x)| = ', lg.norm(grad(x)))
        
          
    print('алгоритм был выполнен ( ', n - 1, ' ) раз')
    return x

## тесты

In [45]:
x0 = np.array([33/13, 4/13, 0.])
print('x0         = ', x0)
print('fx(x0)     = ', fx(x0))
print('H(x0)      = ', Hx(x0))
print('gradfx(x0) = ', gradfx(x0))
print('gradHx(x0) = ', gradHx(x0))

x0         =  [2.53846154 0.30769231 0.        ]
fx(x0)     =  -21.307692307692307
H(x0)      =  0.0
gradfx(x0) =  [-4.92307692 -7.38461538  0.        ]
gradHx(x0) =  [0. 0. 0.]


In [46]:
x0 = np.array([3., 3.7, 2.])
e = 0.05
x_min = sanction(fx, gradfx, Hx, gradHx, x0, e)
print(round(fx(x_min), 3), x_min)

------------------------------------------------------------------------[---- 1 ----]------

------[---- 1 ----]------
  x0    =  [1.33858647 1.19522455 1.16460614]  x1    =  [1.22355316 1.27150281 0.85591771]
  f(x0) =  7.995675397969631  f(x1) =  -2.302067702831639

------[---- 2 ----]------
  x0    =  [1.17368905 0.93528258 1.07357673]  x1    =  [1.09872247 1.07049465 0.78465208]
  f(x0) =  -16.266990204722802  f(x1) =  -16.688183195019505

------[---- 3 ----]------
  x0    =  [1.16151233 0.9031299  1.05755506]  x1    =  [1.09162863 1.04468041 0.77097723]
  f(x0) =  -16.56527789362086  f(x1) =  -16.88469668623369

скачок[---- 0 ----]скачок
a1 =  -3.7603927746334773
  x1  =  [ 0.89872217  1.43541541 -0.02009013]
f(x1) =  -17.50184500471925

------[---- 4 ----]------
  x0    =  [ 0.90728276  1.43144213 -0.02414022]  x1    =  [ 0.79224945  1.50772039 -0.30094338]
  f(x0) =  -17.54652080972912  f(x1) =  -15.342882765737444

------[---- 5 ----]------
  x0    =  [ 0.9156564   1.42726723 -

  x0    =  [ 2.0159922   0.70491664 -0.0594425 ]  x1    =  [ 1.93695894  0.83402869 -0.30086335]
  f(x0) =  -21.08637668548568  f(x1) =  -20.2860291471918

------[---- 87 ----]------
  x0    =  [ 2.02777048  0.69973327 -0.06677426]  x1    =  [ 1.95666449  0.835944   -0.24143838]
  f(x0) =  -21.10814552435728  f(x1) =  -20.494294898855255

скачок[---- 28 ----]скачок
a1 =  0.05478878876265813
  x1  =  [ 2.03166629  0.69227045 -0.05720462]
f(x1) =  -21.10980621262062

------[---- 88 ----]------
  x0    =  [ 2.03500099  0.69190685 -0.05884574]  x1    =  [ 1.91996768  0.76818511 -0.3356489 ]
  f(x0) =  -21.115498050671697  f(x1) =  -19.038802646384962

------[---- 89 ----]------
  x0    =  [ 2.03781787  0.69079166 -0.06063329]  x1    =  [ 1.95878461  0.81990371 -0.30205414]
  f(x0) =  -21.12072192755121  f(x1) =  -20.31967732189926

------[---- 90 ----]------
  x0    =  [ 2.04912874  0.68581399 -0.0676741 ]  x1    =  [ 1.97802275  0.82202472 -0.24233822]
  f(x0) =  -21.140797312782954  f(x1

  x0    =  [ 2.40823517  0.45100792 -0.08067789]  x1    =  [ 2.3280157   0.58142632 -0.32269184]
  f(x0) =  -21.491527880106247  f(x1) =  -20.67513905025154

------[---- 177 ----]------
  x0    =  [ 2.41194068  0.44939448 -0.08302274]  x1    =  [ 2.33987267  0.58672999 -0.25208017]
  f(x0) =  -21.493497931619473  f(x1) =  -20.928619847323233

скачок[---- 58 ----]скачок
a1 =  0.01752261692342244
  x1  =  [ 2.4132035   0.446988   -0.08006041]
f(x1) =  -21.4936655003678

------[---- 178 ----]------
  x0    =  [ 2.41419238  0.44693778 -0.08052276]  x1    =  [ 2.29915907  0.52659665 -0.35732593]
  f(x0) =  -21.49414582658051  f(x1) =  -19.46674867504954

------[---- 179 ----]------
  x0    =  [ 2.41500256  0.44662671 -0.08104265]  x1    =  [ 2.3346758   0.57716327 -0.32311025]
  f(x0) =  -21.494577726636333  f(x1) =  -20.67760635164067

------[---- 180 ----]------
  x0    =  [ 2.41854955  0.44508325 -0.08329001]  x1    =  [ 2.34632682  0.58245987 -0.25241047]
  f(x0) =  -21.496383577049702 

  x0    =  [ 2.52521653  0.3753212  -0.0871164 ]  x1    =  [ 2.44314298  0.50778148 -0.33005739]
  f(x0) =  -21.52544483973589  f(x1) =  -20.69858356203152

------[---- 264 ----]------
  x0    =  [ 2.52627438  0.37486788 -0.08780284]  x1    =  [ 2.45227335  0.51347799 -0.25059608]
  f(x0) =  -21.52559107184025  f(x1) =  -20.984755686614722

скачок[---- 87 ----]скачок
a1 =  0.005114121129069185
  x1  =  [ 2.52665283  0.37415901 -0.08697029]
f(x1) =  -21.5256050737924

------[---- 265 ----]------
  x0    =  [ 2.52691842  0.37416284 -0.08708777]  x1    =  [ 2.41188511  0.45846604 -0.36389093]
  f(x0) =  -21.525638349177587  f(x1) =  -19.588430895811847

------[---- 266 ----]------
  x0    =  [ 2.52712853  0.3740853  -0.08722486]  x1    =  [ 2.44502474  0.50657889 -0.33018097]
  f(x0) =  -21.525667432091918  f(x1) =  -20.698622156442696

------[---- 267 ----]------
  x0    =  [ 2.52813681  0.37365331 -0.08787939]  x1    =  [ 2.45409082  0.51227281 -0.25069066]
  f(x0) =  -21.52580029794230

------------------------------------------------------------------------[---- 15 ----]------

------[---- 1 ----]------
  x0    =  [ 2.55225052  0.34579181 -0.07148867]  x1    =  [ 2.37279035  0.33402755 -0.38539752]
  f(x0) =  -21.479989689601187  f(x1) =  -10.988313055389149

------[---- 2 ----]------
  x0    =  [ 2.55223823  0.3457202  -0.07143014]  x1    =  [ 2.44862606  0.44694206 -0.33099873]
  f(x0) =  -21.47999475753231  f(x1) =  -19.807773681525678

------[---- 3 ----]------
  x0    =  [ 2.55224708  0.34566628 -0.0713477 ]  x1    =  [ 2.4710255   0.47948084 -0.30242652]
  f(x0) =  -21.4799996259221  f(x1) =  -20.511173714635802
сделано  3  шагов
сделано  0  скачков
 -------------------------------------------------------------- f(x) + ((  17.61577310586391  )) * H(x)    =     -21.478001712911055
 --------------------------------------------------------------   x       =  [ 2.55224708  0.34566628 -0.0713477 ]
 -------------------------------------------------------------- H(x) 

In [31]:
x0 = np.array([33/13, 4/13, 0])
e = 0.05
x_min = sanction(fx, gradfx, Hx, gradHx, x0, e)
print(round(fx(x_min), 3), x_min)

------------------------------------------------------------------------[---- 1 ----]------

------[---- 1 ----]------
  x0    =  [2.54999901 0.32499851 0.        ]  x1    =  [ 2.4349657   0.41354033 -0.27680317]
  f(x0) =  -21.415341128020838  f(x1) =  -20.13940058416887

------[---- 2 ----]------
  x0    =  [ 2.55191764  0.32787646 -0.00478238]  x1    =  [ 2.46819206  0.46215611 -0.24854938]
  f(x0) =  -21.42907784706481  f(x1) =  -21.15798627927266

------[---- 3 ----]------
  x0    =  [ 2.55474291  0.33211436 -0.01988998]  x1    =  [ 2.47546083  0.46935507 -0.21475674]
  f(x0) =  -21.461466410971187  f(x1) =  -21.250726936496665

скачок[---- 0 ----]скачок
a1 =  -0.3207982554387822
  x1  =  [ 2.52930936  0.37614094 -0.0824029 ]
f(x1) =  -21.52197782984626

------[---- 4 ----]------
  x0    =  [ 2.527544    0.37310773 -0.08382088]  x1    =  [ 2.41251069  0.45728455 -0.36062405]
  f(x0) =  -21.525366628480842  f(x1) =  -19.61985267602204

------[---- 5 ----]------
  x0    =  [ 2.52757

  x0    =  [ 2.55212768  0.34771546 -0.07451701]  x1    =  [ 2.44374524  0.44404879 -0.34720217]
  f(x0) =  -21.48639925496292  f(x1) =  -19.524576197486546

------[---- 2 ----]------
  x0    =  [ 2.55213357  0.34764873 -0.07441099]  x1    =  [ 2.47020786  0.4826652  -0.3163674 ]
  f(x0) =  -21.486407074918134  f(x1) =  -20.461686435311748

------[---- 3 ----]------
  x0    =  [ 2.55226085  0.34726138 -0.07360826]  x1    =  [ 2.47352803  0.47939862 -0.19444832]
  f(x0) =  -21.48645043511912  f(x1) =  -20.976785147511837
сделано  3  шагов
сделано  0  скачков
 -------------------------------------------------------------- f(x) + ((  17.0  )) * H(x)    =     -21.48412068765223
 --------------------------------------------------------------   x       =  [ 2.55226085  0.34726138 -0.07360826]
 -------------------------------------------------------------- H(x)      =  0.010703116122008543
 -------------------------------------------------------------- |grad(x)| =  0.12364355299001333
-------